In [1]:
import numpy as np
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))
class GF():
    def __init__(self, f):
        Max = np.max(f)
        f2 = np.zeros(Max+1, np.uint8)
        f2[f] = 1
        f = np.flip(f2)
        self.len_of_symbol = Max
        self.q = 2**self.len_of_symbol
        self.table = np.array(self.make_code_table(f))
        
    def sub_matrix(self, A, i, j):
        A1 = np.delete(A, i, 0)
        A2 = np.delete(A1, j, 1)
        return A2

    def det(self, A):
        size = A.shape
        if size[0]==2 and size[1]==2:
            d = self.add(self.multi(A[0, 0],A[1, 1]), self.multi(A[1, 0],A[0, 1]))
            return d
        else:
            d = 0
            for i in range(size[0]):
                d=self.add(self.multi(A[0, i],self.det(self.sub_matrix(A, 0, i))), d)
            return d
        
    def make_code_table(self, f):
        m = len(f)-1
        N = 2**m
        table = ["0"*m]
        x =""
        for i in range(1,m+1):
            if f[i]==0:
                x+="0"
            else:
                x+="1"
        for i in range(0, m):
            c="0"*m
            c = c[0:i] + "1"+c[i+1:]
            c = c[::-1]
            table.append(c)
        for i in range(m+1, N):
            a = table[-1]
            a1 = a[1:]+"0"
            b = "0"*m
            if a[0]=="1":
                b = x
            c = self.add_string_code(a1, b)
            table.append(c)
        return table

    def add_string_code(self, a, b):
        c =""
        for i in range(self.len_of_symbol):
            if a[i]!= b[i] :
                c+="1"
            else:
                c+="0"
        return c
    def add(self, a, b):
        if  np.isscalar(a) and  np.isscalar(b):
            a= int(a)
            b = int(b)
            a1 = self.table[a]
            b1 = self.table[b]
            c1 = self.add_string_code(a1, b1)
            c, = np.where(self.table == c1)
            return c[0]
        elif not np.isscalar(a):
            if np.isscalar(b):
                raise ValueError("a, b must have same shape ( a:",a.shape,", b is scalar)" )
            else:
                a_size = a.shape
                b_size = b.shape
                if  len(a_size) ==2 and  len(a_size)==2:
                    if a_size[0] != b_size[0] or a_size[1] != b_size[1] :
                        raise ValueError("a, b must have same shape ( a:"+ str(a.shape) +",  b:"+str(b.shape)+ ")" )
                    else:
                        c = np.zeros_like(a, np.uint8)
                        for i in range(a_size[0]):
                            for j in range(a_size[1]):
                                c[i, j] = self.add(a[i, j], b[i, j])
                        return c
                elif len(a_size)==1 and len(b_size)==1 and a_size ==b_size :
                    c = np.zeros_like(a, np.uint8)
                    for i in range(a_size[0]):
                        c[i] = self.add(a[i], b[i])
                    return c
                else:
                    raise ValueError("a, b must have same shape ( a:"+ str(a.shape) +",  b:"+str(b.shape)+ ")" )         
    def multi(self, a, b):
        if  np.isscalar(a) and  np.isscalar(b):
                if a ==0 or b == 0:
                    return 0
                else:
                    c = (a+b-2) % (self.q-1)
                    return c+1
        elif not np.isscalar(a):
            a_size = a.shape
            if np.isscalar(b):
                c = [self.multi(i, b) for i in a]
                return np.array(c)
            b_size = b.shape
            if a_size[1] != b_size[0] :
                raise ValueError("Can not multiply 2 matrix of dimension: ", a.shape, b.shape)
            else:
                c = np.zeros((a_size[0], b_size[1]), np.uint8)
                for i in range(a_size[0]):
                    for j in range(b_size[1]):
                        for k in range(a_size[1]):
                            c[i, j] =self.add(c[i, j], self.multi(a[i, k], b[k, j])) 
                return c
    def make_sub_matrix(self, x):
        N =  N = self.q -1
        sub = np.zeros((N, N), np.uint8)
        index = x-1
        for i in range(N):
            sub[i, index] = index+1
            index +=1
            if index ==N:
                index =0
        return sub
    def reserve_e(self, a):
            a2 = self.q+1-a
            if a2 == self.q:
                a2 = 1
            if a2 == self.q+1:
                a2 = 0
            return a2
    def Reserve(self, X, show =False):
        N = X.shape[0]
        def make_I(n):
                In = np.zeros((n, n), np.uint8)
                for i in range(n):
                    In[i, i] =1
                return In
        X1 = make_I(N)
        for i in range(N):
            if show:
                print(i)
            C = make_I(N)
            for j in range(N):
                C[j, i] = self.multi(X[j, i], self.reserve_e(X[i,i]))
            C[i,i] = self.reserve_e(X[i,i])
            X = self.multi(C,X)
            X1 = self.multi(C,X1)
        return X1
    
import numpy as np
from scipy.special import erfc

def S2B(symbol, table):
    b = table[symbol]
    B = np.array(list(b)).astype(np.float32)
    return B

def probB(x, N0):
    d1 = abs(-1-x)
    d2 = abs(1-x)
    p0 = np.e**(-d1/N0)/(np.e**(-d1/N0)+np.e**(-d2/N0))
    p1 = 1-p0
    return [p0, p1]

def probS(S, N0, q, table, M):
    B = np.zeros((q, 2))
    for i in range(q):
        B[i] = probB(S[i], N0)
    P = np.zeros(M)
    for i in range(M):
        Bi = S2B(i, table)
        p = 1
        for j in range(q):
            p *=B[j, int(Bi[j])]
        P[i] = p
    m = np.max(P)
    P = np.log10(m/P)
    P[P>2]=2
    return P

def BPSK(message, M, table, SNRdB):
    Eb = 1
    N0 = Eb/10**(SNRdB/10)
    q = int(np.log2(M))
    Ns = len(message)
    Nb = Ns*q
    noise = np.sqrt(N0/2)*np.random.randn(Nb)
    noise = noise.reshape((Ns, q))
    X = np.zeros_like(noise)
    for i in range(Ns):
        X[i]= S2B(message[i], table)
    X[X==0]=-1
    Y = X+noise
    output = np.zeros((Ns, M))
    for i in range(Ns):
        output[i] = probS(Y[i], N0, q, table, M)
    return output

def HardDicision(Message):
    return np.argmin(Message, axis=1)

def score(X, Y):
    Z = np.zeros_like(X)
    Z[X!=Y]=1
    return np.sum(Z)

def predict(Q):
    message = np.argmin(Q, axis=0)
    X = 0
    for i in range(dc):
        X = gl.add(X, message[i])
    return message, X

def permutation(X, hi):
    Y = np.zeros_like(X)
    Y[0] = X[0]
    for i in range(1, X.shape[0]):
        Y[gl.multi(i, hi)] = X[i]
    return Y

def permutation2d(X, Y):
    Z = np.zeros_like(X)
    for i in range(X.shape[1]):
        Z[:, i] = permutation(X[:, i], Y[i])
    return Z

def unpermutation2d(X, Y):
    Z = np.zeros_like(X)
    for i in range(X.shape[1]):
        Z[:, i] = permutation(X[:, i], gl.reserve_e(Y[i]))
    return Z

def convert_vhdl(A, elem32=False):
    shape = A.shape
    s = "("
    if len(shape)==1:
        for i in A:
            si = int(i)
            s+=str(si)+", "
        s=s[:-2]+")"
        return s
    if len(shape)==2:
        for row in A:
            print(convert_vhdl(row)+",")

def debug(name, value, debug=False):
    if debug:
        print(name+"#"*30)
        print(value)
        
def row_selection(Q, H_i, H_non_zeros_index_i):
    Qi = np.zeros((q, dc))
    Qi = Q[:,H_non_zeros_index_i]
    Qi = permutation2d(Qi, H_i[H_non_zeros_index_i])
    return Qi

def row_update(Q, Qi2, H_i, H_non_zeros_index_i):
    Qi2 = unpermutation2d(Qi2,H_i[H_non_zeros_index_i])
    Q[:, H_non_zeros_index_i] = Qi2
    return Q
    
def CN(Q, A= 0.5,  dg=False):
    #print(convert_vhdl(Q))
    zn = np.argmin(Q, axis=0)
    debug("Zn", zn, dg)
    dQ = np.zeros_like(Q)
    for j in range(dc):
        for a in range(q):
            nj = gl.add(a, zn[j])
            dQ[nj, j] = Q[a, j]
            
    debug("dQ", dQ, dg)
    B = 0
    for i in range(dc):
        B = gl.add(B, zn[i])
    debug("B", B, dg)
    def sai(A):
        col = np.argmin(A)
        sortA = np.sort(A)
        return sortA[0], sortA[1], col

    m = np.zeros((q-1, 3))
    for i in range(1, q):
        m[i-1]= sai(dQ[i])
    debug("m", m, dg)
    debug("m0", m[:, 0], dg)
#     print(convert_vhdl(m[:, 0]))        
#     print("m1"+"#"*30)
#     print(convert_vhdl(m[:, 1], True))
    conf = np.ones((q, q))*100000
    conf[0, 0] = 0
    conf[1:, 0] = m[:, 0]
    for i in range(1,q-1):
        for j in range(i+1, q):
            if m[i-1, 2]!=m[j-1,2]:
                conf[gl.add(i,j), i] = max(m[i-1, 0],m[j-1,0])
    debug("conf", conf, dg)
    extra_columm = np.zeros((q, 6))
    extra_columm[:, 0] = np.min(conf, axis=1)
    extra_columm[1:, 1] = m[:, 0]
    extra_columm[1:, 2] = m[:, 1]
    extra_columm[1:, 5] = m[:, 2]
    argmin=  np.argmin(conf, axis=1)
    for i in range(1, q):
        if argmin[i]==0:
            extra_columm[i, 3] = extra_columm[i, 4] = m[i-1, 2]
        else:
            d1 = m[argmin[i]-1, 2]
            d2 = m[gl.add(argmin[i], i)-1, 2]
            extra_columm[i, 3]=d1
            extra_columm[i, 4] =d2
    debug("extra_columm", extra_columm, dg)
    dR = np.zeros_like(dQ)
    
    def dRij(extra_colummi, j):
        dQi, m1, m2, d1, d2, mcol = extra_colummi
        if d1==j or d2==j:
            if d1 == d2:
                return m2
            else:
                if mcol ==j:
                    return m2
                else:
                    return m1
        else:
            return dQi
        
    for i in range(q):
        for j in range(dc):
            dR[i, j] = dRij(extra_columm[i], j)
    debug("dR", dR, dg)
    R = np.zeros_like(Q)
    for i in range(q):
        for j in range(dc):
            index = gl.add(i, B)
            index = gl.add(index, zn[j])
            R[index, j]= (A*dR[i,j]).astype(dtype)
            
    debug("R", R,dg) 
    return R

def quick_check(H, C):
    C = C.reshape((C.shape[0], 1))
    return gl.multi(np.array(H), C)

gl = GF([3,  1, 0])
print(gl.table)

['000' '001' '010' '100' '011' '110' '111' '101']


In [6]:
Scale = 50
dc = 5
q = 8
dtype = np.int16
#dtype = np.float32
G = np.load("G.npy").astype(np.int16)
H = np.load("H.npy").astype(np.int16)
sub = np.load("sub.npy").astype(np.int16)
print(sub)
m = H.shape[0]
n = H.shape[1]
k = n- m
print(m, n, k)
H_non_zeros_index  = np.zeros((m, dc), np.uint16)
for i in range(m):
    stt = 0
    for j in range(n):
        if H[i, j]!=0:
            H_non_zeros_index[i, stt] = j
            stt+=1           

    
for i in range(m):
    Hi = H_non_zeros_index[i]
    Hi2 = np.zeros_like(Hi)
    for j in range(dc):
        Hi2[j] = Hi[j]- j*(q-1)
    print(convert_vhdl(Hi2))
print("#"*20)
for i in range(m):
    print(convert_vhdl(H_non_zeros_index[i]))
print("#"*20)
for i in range(m):
    print(convert_vhdl(H[i][H_non_zeros_index[i]]))

[25 24 34 22 27 23  6  7  8  9 10 32 12 13 14 15 16 17 18 19 20 21  3  5
 26 28 29  4  0 30 31  1 33 11  2]
13 35 22
(0, 3, 5, 2, 1)
(6, 1, 5, 4, 2)
(5, 1, 3, 0, 6)
(2, 5, 0, 4, 3)
(1, 4, 6, 3, 2)
(6, 2, 4, 1, 0)
(5, 0, 4, 3, 1)
(4, 0, 2, 6, 5)
(3, 6, 1, 5, 4)
(1, 3, 0, 6, 4)
(3, 5, 2, 1, 6)
(4, 6, 3, 2, 0)
(0, 2, 6, 5, 3)
####################
(0, 10, 19, 23, 29)
(6, 8, 19, 25, 30)
(5, 8, 17, 21, 34)
(2, 12, 14, 25, 31)
(1, 11, 20, 24, 30)
(6, 9, 18, 22, 28)
(5, 7, 18, 24, 29)
(4, 7, 16, 27, 33)
(3, 13, 15, 26, 32)
(1, 10, 14, 27, 32)
(3, 12, 16, 22, 34)
(4, 13, 17, 23, 28)
(0, 9, 20, 26, 31)
####################
(1, 4, 6, 3, 2)
(7, 2, 6, 5, 3)
(6, 2, 4, 1, 7)
(3, 6, 1, 5, 4)
(2, 5, 7, 4, 3)
(7, 3, 5, 2, 1)
(6, 1, 5, 4, 2)
(5, 1, 3, 7, 6)
(4, 7, 2, 6, 5)
(2, 4, 1, 7, 5)
(4, 6, 3, 2, 7)
(5, 7, 4, 3, 1)
(1, 3, 7, 6, 4)


In [3]:
print(gl.add(6, 4))

3


In [8]:
for SNRdB in range(6):
    print(SNRdB)
    samples = int(1e6)
    Y = np.zeros((samples, n), np.uint8)
    X = np.zeros((samples,n, q), np.float16)
    for i in range(samples):
        u = np.random.randint(0, q, k).reshape(k, 1)
        p = gl.multi(G, u)
        c0 = np.concatenate((u, p))
        c = np.zeros_like(c0)
        c[sub.astype(np.uint8)] = c0
        c = c.reshape(n)
        Q0 = BPSK(c, q, gl.table, SNRdB)
        Y[i]= c.copy()
        X[i]=Q0.copy()
    np.save("X"+str(SNRdB)+".npy", X)
    np.save("Y"+str(SNRdB)+".npy", Y)


0
1
2
3
4
5


In [11]:
from tqdm import tnrange
import time
def decoder2(Q, A):
    Rmna = np.zeros((m, q, dc), dtype)
    citer = 0
    for t in range(5):
        citer+=1
        stt = 0
        for i in range(m):
#             print(i)
#             print( H_non_zeros_index[i])
#             print(H[i][H_non_zeros_index[i]])
            Qmnai = permutation2d(Q[:, H_non_zeros_index[i]], H[i][H_non_zeros_index[i]])-Rmna[i]
#             convert_vhdl( Q[:, H_non_zeros_index[i]].transpose())
#             print("#"*20)
            minQ = np.min(Qmnai, axis=0)
            Qmnai = Qmnai - minQ
#             convert_vhdl( Qmnai.transpose())
#             print("#"*20)
            Rmna[i] = CN(Qmnai, A)
#             convert_vhdl( Rmna[i].transpose())
            QRmni = Rmna[i]+ Qmnai
            Q[:, H_non_zeros_index[i]] = unpermutation2d(QRmni,  H[i][H_non_zeros_index[i]])
        y, s = predict(Q)
        s = np.sum(quick_check(H, y))
        if s == 0:
            break
    return y,citer

def test(N, A, SNRdB):
    NN = 0
    EE = 0
    E0 = 0
    EN = 0
    EN0 = 0 
    for i in tnrange(N):
        t1= time.time()
        u = np.random.randint(0, q, k).reshape(k, 1)
        p = gl.multi(G, u)
        c0 = np.concatenate((u, p))
        c = np.zeros_like(c0)
        c[sub.astype(np.uint8)] = c0
        c = c.reshape(n)
        Q0 = BPSK(c, q, gl.table, SNRdB)
        t2= time.time()
        Q0 = Q0.transpose()
        _, s = predict(Q0)
        e0 = score(c, _)
        E0+=e0
        if e0!=0:
            EN0+=1
        t3= time.time()
        Q = (Q0*Scale).astype(np.int16)
        y,inters = decoder2(Q0, A)
        t4= time.time()
        #print(t4-t1)
        e = score(c, y)
        EE+=e
        NN+=n
        if e!=0:
            EN+=1
        
    return NN, E0, EE, EN0, EN
    


# NN, E0,  EE, EN0, EN = test(1000, 1, 6)
# print(NN, E0, EN0, EE, EN)

NN, E0,  EE, EN0, EN = test(500000, 1, 5.6)
print(NN, E0, EN0, EE, EN)

# NN, E0,  EE, EN0, EN = test(10000, 1, 5)
# print(NN, E0, EN0, EE, EN)

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:36: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`



17500000 184457 155112 10 8


In [ ]:
SNRdbs = [4.2, 4.4, 4.6, 4.8, 5, 5.2]
for SNRdb in SNRdbs:
    S = 0
    E = 0
    while S<1e8 and E<5 :
        NN, E0,  EE, EN0, EN = test(100, 1, SNRdb)
        S+=NN
        E+=EE
    print(S, E, np.log10(E/S))

# 4.2 1990800 6 -5.520876381688342
# 4.4 2022300 5 -5.606875577522435
# 4.6 2990400 6 -5.697578033651113
# 4.8 8769600 5 -6.244009780392107

In [4]:
u = np.random.randint(0, q, k).reshape(k, 1)
p = gl.multi(G, u)
c0 = np.concatenate((u, p))
c = np.zeros_like(c0)
c[sub.astype(np.uint8)] = c0
c = c.reshape(n)
print(c)
Q0 = BPSK(c, q, gl.table, 3)
Q0 = Q0.transpose()
Q = (Q0*Scale).astype(np.int16)

[1 5 4 0 3 7 4 6 0 5 6 7 4 5 7 3 7 2 4 6 4]


In [5]:
print(Q)

[[ 86  96  76   0  61 100 100 100   0 100 100  92 100 100 100  86  94  75
  100 100 100]
 [  0 100  10  86  88  86  86 100  86 100 100  60  86 100  86 100  86 100
   80 100  14]
 [100  15  65  86 100 100  86 100  18  76 100 100  86  86 100 100 100   0
   75 100  92]
 [ 95  80 100  78   0  86 100 100  86  60 100  31 100  86  77   0   7 100
  100  96  95]
 [ 86  80   0 100 100 100   0  61 100  93  74 100   0 100 100 100 100  86
    0  86   5]
 [100   0 100 100  86 100 100  73 100   0  71 100 100   0 100  86  94  49
  100  46  86]
 [ 95  64  86 100 100  85  54   0 100  16   0  73  86  81  86 100  86 100
   57   0   0]
 [  8 100  97 100  27   0 100  86 100  77  86   0 100 100   0  86   0 100
  100  50   8]]


In [15]:
import time
from bitstring import BitArray
stt=0

import numpy as np
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))
class GF():
    def __init__(self, f):
        Max = np.max(f)
        f2 = np.zeros(Max+1, np.uint8)
        f2[f] = 1
        f = np.flip(f2)
        self.len_of_symbol = Max
        self.q = 2**self.len_of_symbol
        self.table = np.array(self.make_code_table(f))
        
    def sub_matrix(self, A, i, j):
        A1 = np.delete(A, i, 0)
        A2 = np.delete(A1, j, 1)
        return A2

    def det(self, A):
        size = A.shape
        if size[0]==2 and size[1]==2:
            d = self.add(self.multi(A[0, 0],A[1, 1]), self.multi(A[1, 0],A[0, 1]))
            return d
        else:
            d = 0
            for i in range(size[0]):
                d=self.add(self.multi(A[0, i],self.det(self.sub_matrix(A, 0, i))), d)
            return d
        
    def make_code_table(self, f):
        m = len(f)-1
        N = 2**m
        table = ["0"*m]
        x =""
        for i in range(1,m+1):
            if f[i]==0:
                x+="0"
            else:
                x+="1"
        for i in range(0, m):
            c="0"*m
            c = c[0:i] + "1"+c[i+1:]
            c = c[::-1]
            table.append(c)
        for i in range(m+1, N):
            a = table[-1]
            a1 = a[1:]+"0"
            b = "0"*m
            if a[0]=="1":
                b = x
            c = self.add_string_code(a1, b)
            table.append(c)
        return table

    def add_string_code(self, a, b):
        c =""
        for i in range(self.len_of_symbol):
            if a[i]!= b[i] :
                c+="1"
            else:
                c+="0"
        return c
    def add(self, a, b):
        if  np.isscalar(a) and  np.isscalar(b):
            a= int(a)
            b = int(b)
            a1 = self.table[a]
            b1 = self.table[b]
            c1 = self.add_string_code(a1, b1)
            c, = np.where(self.table == c1)
            return c[0]
        elif not np.isscalar(a):
            if np.isscalar(b):
                raise ValueError("a, b must have same shape ( a:",a.shape,", b is scalar)" )
            else:
                a_size = a.shape
                b_size = b.shape
                if  len(a_size) ==2 and  len(a_size)==2:
                    if a_size[0] != b_size[0] or a_size[1] != b_size[1] :
                        raise ValueError("a, b must have same shape ( a:"+ str(a.shape) +",  b:"+str(b.shape)+ ")" )
                    else:
                        c = np.zeros_like(a, np.uint8)
                        for i in range(a_size[0]):
                            for j in range(a_size[1]):
                                c[i, j] = self.add(a[i, j], b[i, j])
                        return c
                elif len(a_size)==1 and len(b_size)==1 and a_size ==b_size :
                    c = np.zeros_like(a, np.uint8)
                    for i in range(a_size[0]):
                        c[i] = self.add(a[i], b[i])
                    return c
                else:
                    raise ValueError("a, b must have same shape ( a:"+ str(a.shape) +",  b:"+str(b.shape)+ ")" )         
    def multi(self, a, b):
        if  np.isscalar(a) and  np.isscalar(b):
                if a ==0 or b == 0:
                    return 0
                else:
                    c = (a+b-2) % (self.q-1)
                    return c+1
        elif not np.isscalar(a):
            a_size = a.shape
            if np.isscalar(b):
                c = [self.multi(i, b) for i in a]
                return np.array(c)
            b_size = b.shape
            if a_size[1] != b_size[0] :
                raise ValueError("Can not multiply 2 matrix of dimension: ", a.shape, b.shape)
            else:
                c = np.zeros((a_size[0], b_size[1]), np.uint8)
                for i in range(a_size[0]):
                    for j in range(b_size[1]):
                        for k in range(a_size[1]):
                            c[i, j] =self.add(c[i, j], self.multi(a[i, k], b[k, j])) 
                return c
    def make_sub_matrix(self, x):
        N =  N = self.q -1
        sub = np.zeros((N, N), np.uint8)
        index = x-1
        for i in range(N):
            sub[i, index] = index+1
            index +=1
            if index ==N:
                index =0
        return sub
    def reserve_e(self, a):
            a2 = self.q+1-a
            if a2 == self.q:
                a2 = 1
            if a2 == self.q+1:
                a2 = 0
            return a2
    def Reserve(self, X, show =False):
        N = X.shape[0]
        def make_I(n):
                In = np.zeros((n, n), np.uint8)
                for i in range(n):
                    In[i, i] =1
                return In
        X1 = make_I(N)
        for i in range(N):
            if show:
                print(i)
            C = make_I(N)
            for j in range(N):
                C[j, i] = self.multi(X[j, i], self.reserve_e(X[i,i]))
            C[i,i] = self.reserve_e(X[i,i])
            X = self.multi(C,X)
            X1 = self.multi(C,X1)
        return X1
    


gl = GF([3,  1, 0])
print(gl.table)

def int2bits(x):
    return '{0:08b}'.format(x)

def int2creal(x):
    return '{0:010b}'.format(x)

def byte2int(byte):
    return int(byte, 2)

def split_with_length(bits, length):
    res=[bits[y-length:y] for y in range(length, len(bits)+length,length)]
    return res

def bits2frame(bits):
    bytearrays = split_with_length(bits[:63], 3)
    symbols = [byte2int(s) for s in bytearrays]
    return symbols

def bits2subQ(bits):
    bytearrays = split_with_length(bits, 10)
    Q = [byte2int(s) for s in bytearrays]
    Q = np.array(Q).reshape((3, 8))
    return Q

def bits2Q(bits):
    bytearrays = split_with_length(bits, 10)
    Q = [byte2int(s) for s in bytearrays]
    Q = np.array(Q).reshape((21, 8))
    return Q

def send(Q):
    ser.write(bytes([15]))
    Q2 = Q.transpose()
    print(Q2)
    print("#"*20)
    A =Q2.reshape(n*q)
    print(A)
    print("#"*20)
    B = [int2creal(x) for x in A]
    S=""
    for s in B:
        S+=s
    bytes_arr = split_with_length(S, 8)
    for byte in bytes_arr:
        ser.write(bytes([int(byte, 2)]))
        time.sleep(0.0001)

['000' '001' '010' '100' '011' '110' '111' '101']


In [7]:
import serial
from serial import Serial
ser = Serial('COM4', 115200, serial.EIGHTBITS, serial.PARITY_NONE, serial.STOPBITS_ONE)

In [8]:
def Listening():
    print("listening!")
    data=""
    stt=0
    while cf.RUN:
        res = ser.read()
        data+=int2bits(ord(res))
        stt+=1
        if stt ==8:
            print(data)
            print(bits2frame(data))
            data = ""
            stt = 0
    print("stoped")
    
def Listening1():
    print("listening!")
    data=""
    stt=0
    while cf.RUN:
        res = ser.read()
        data+=int2bits(ord(res))
        stt+=1
        if stt ==30:
            print(bits2subQ(data))
            data = ""
            stt = 0
    print("stoped")
    
def Listening2():
    print("listening!")
    data=""
    stt=0
    while cf.RUN:
        res = ser.read()
        data+=int2bits(ord(res))
        stt+=1
        if stt ==210:
            print(bits2Q(data))
            print("#"*20)
            data = ""
            stt = 0
    print("stoped")

In [9]:
import config as cf
import threading
cf.RUN =True
listen = threading.Thread(target=Listening)
listen.start()

listening!


In [17]:
send(Q)

[[ 86   0 100  95  86 100  95   8]
 [ 96 100  15  80  80   0  64 100]
 [ 76  10  65 100   0 100  86  97]
 [  0  86  86  78 100 100 100 100]
 [ 61  88 100   0 100  86 100  27]
 [100  86 100  86 100 100  85   0]
 [100  86  86 100   0 100  54 100]
 [100 100 100 100  61  73   0  86]
 [  0  86  18  86 100 100 100 100]
 [100 100  76  60  93   0  16  77]
 [100 100 100 100  74  71   0  86]
 [ 92  60 100  31 100 100  73   0]
 [100  86  86 100   0 100  86 100]
 [100 100  86  86 100   0  81 100]
 [100  86 100  77 100 100  86   0]
 [ 86 100 100   0 100  86 100  86]
 [ 94  86 100   7 100  94  86   0]
 [ 75 100   0 100  86  49 100 100]
 [100  80  75 100   0 100  57 100]
 [100 100 100  96  86  46   0  50]
 [100  14  92  95   5  86   0   8]]
####################
[ 86   0 100  95  86 100  95   8  96 100  15  80  80   0  64 100  76  10
  65 100   0 100  86  97   0  86  86  78 100 100 100 100  61  88 100   0
 100  86 100  27 100  86 100  86 100 100  85   0 100  86  86 100   0 100
  54 100 100 100 100 100

Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\program files\python37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "c:\program files\python37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-8-43736ccda6d1>", line 36, in Listening2
    data+=int2bits(ord(res))
TypeError: ord() expected a character, but string of length 0 found



In [ ]:
cf.RUN =False

In [13]:
convert_vhdl(Q.transpose())

(86, 0, 100, 95, 86, 100, 95, 8),
(96, 100, 15, 80, 80, 0, 64, 100),
(76, 10, 65, 100, 0, 100, 86, 97),
(0, 86, 86, 78, 100, 100, 100, 100),
(61, 88, 100, 0, 100, 86, 100, 27),
(100, 86, 100, 86, 100, 100, 85, 0),
(100, 86, 86, 100, 0, 100, 54, 100),
(100, 100, 100, 100, 61, 73, 0, 86),
(0, 86, 18, 86, 100, 100, 100, 100),
(100, 100, 76, 60, 93, 0, 16, 77),
(100, 100, 100, 100, 74, 71, 0, 86),
(92, 60, 100, 31, 100, 100, 73, 0),
(100, 86, 86, 100, 0, 100, 86, 100),
(100, 100, 86, 86, 100, 0, 81, 100),
(100, 86, 100, 77, 100, 100, 86, 0),
(86, 100, 100, 0, 100, 86, 100, 86),
(94, 86, 100, 7, 100, 94, 86, 0),
(75, 100, 0, 100, 86, 49, 100, 100),
(100, 80, 75, 100, 0, 100, 57, 100),
(100, 100, 100, 96, 86, 46, 0, 50),
(100, 14, 92, 95, 5, 86, 0, 8),


In [ ]:
decoder2(Q, 1)

In [ ]:
subQ = Q[:, H_non_zeros_index[0]]
convert_vhdl(subQ)
R = CN(subQ, 1, True)

In [ ]:
y, citer = decoder2(Q.copy(), 1)
print(citer)

In [ ]:
print(y)

In [ ]:
print(gl.add(6, 3))
print(Q[4, 0])

In [ ]:
R = CN(Q, True)

In [ ]:
def convert_vhdl(A, elem32=False):
    shape = A.shape
    s = "("
    if len(shape)==1:
        for i in A:
            si = int(i)
            s+=str(si)+", "
        s=s[:-2]+")"
        return s
    if len(shape)==2:
        for row in A:
            print(convert_vhdl(row)+",")
            
c = np.random.randint(0, q, dc)
Q0 = BPSK(c, q, gl.table, 4)
Q0 = Q0.transpose()
Q = (Q0*1000).astype(np.int16)
convert_vhdl(Q)

In [ ]:
import cv2
_H = 160
_W = 120
def int2Bits(x):
    B = '{0:08b}'.format(x)
    B = np.array(list(B)).astype(np.int8)
    return B

def img_to_bit(img):
    img = cv2.resize(img, (_H, _W))
    cv2.imshow("img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    data = np.zeros(W*H*8*3)
    for i in range(W):
        for j in range(H):
            data[(i*H+j)*24:(i*H+j+1)*24]=np.concatenate((int2Bits(img[i,j, 0]), int2Bits(img[i,j, 1]), int2Bits(img[i,j, 2])))
    return data
                                                         
img = cv2.imread("m.jpg")
data = img_to_bit(img)

In [ ]:
lob = gl.len_of_symbol
text = "hello everyone, this is NB-LPDC simulink program by De Lai"
def text_to_bits(text, encoding='utf-8', errors='surrogatepass'):
    bits = bin(int.from_bytes(text.encode(encoding, errors), 'big'))[2:]
    return bits.zfill(8 * ((len(bits) + 7) // 8))

def text_from_bits(bits, encoding='utf-8', errors='surrogatepass'):
    result = ""
    bytes_splited = split_with_length(bits, 8)
    for byte in bytes_splited:
        n = int(byte, 2)
        b = "0"
        try:
            b = n.to_bytes((n.bit_length() + 7) // 8, 'big').decode(encoding, errors) or '\0'
        except:
            pass
        result+=b
    return result

def split_with_length(bits, length):
    res=[bits[y-length:y] for y in range(length, len(bits)+length,length)]
    return res

def text_to_frames(text):
    bits = text_to_bits(text)
    l = len(bits)
    l_pad = l if l%(lob*k)==0 else lob*k*(l//(lob*k)+1)
    n_pad = l_pad-l
    bits+="0"*n_pad
    S = split_with_length(bits, lob)
    symbols = [np.where(gl.table==s)[0][0] for s in S]
    num_frame = l_pad//(k*lob)
    frames= np.array(symbols).reshape((num_frame, k))
    return frames, l

def bits_to_frames(bits):
    l = len(bits)
    l_pad = l if l%(lob*k)==0 else lob*k*(l//(lob*k)+1)
    n_pad = l_pad-l
    bits+="0"*n_pad
    S = split_with_length(bits, lob)
    symbols = [np.where(gl.table==s)[0][0] for s in S]
    num_frame = l_pad//(k*lob)
    frames= np.array(symbols).reshape((num_frame, k))
    return frames, l

def frames_to_text(frames, l):
    symbols = frames.reshape(frames.shape[0]*frames.shape[1])
    S = [gl.table[s] for s in symbols]
    bits = ""
    for s in S:
        bits+=s
    bits = bits[:l]
    text = text_from_bits(bits)
    return text

def frames_to_bits(frames, l):
    symbols = frames.reshape(frames.shape[0]*frames.shape[1])
    S = [gl.table[s] for s in symbols]
    bits = ""
    for s in S:
        bits+=s
    bits = bits[:l]
    return bits
    
def send_text(text, SNRdB):
    frames, l = text_to_frames(text)
    received_frames = np.zeros_like(frames)
    decoded_frames = np.zeros_like(frames)
    for i in range(frames.shape[0]):
        u =frames[i].reshape(k, 1)
        p = gl.multi(G, u)
        c0 = np.concatenate((u, p))
        c = c0.reshape(n)
        Q0 = BPSK(c, q, gl.table, SNRdB)
        Q0 = Q0.transpose()
        y, s = predict(Q0)
        received_frames[i] = y[:k]
        y2 = decoder2(Q0)
        decoded_frames[i] = y2[:k]
    decoded_text = frames_to_text(decoded_frames, l)
    received_text = frames_to_text(received_frames, l)
    return decoded_text, received_text

def send_bits(text, SNRdB):
    frames, l = bits_to_frames(text)
    received_frames = np.zeros_like(frames)
    decoded_frames = np.zeros_like(frames)
    for i in range(frames.shape[0]):
        u =frames[i].reshape(k, 1)
        p = gl.multi(G, u)
        c0 = np.concatenate((u, p))
        c = c0.reshape(n)
        Q0 = BPSK(c, q, gl.table, SNRdB)
        Q0 = Q0.transpose()
        y, s = predict(Q0)
        received_frames[i] = y[:k]
        y2 = decoder2(Q0)
        decoded_frames[i] = y2[:k]
    decoded_bits = frames_to_bits(decoded_frames, l)
    received_bits = frames_to_bits(received_frames, l)
    return decoded_bits, received_bits
    

In [ ]:
send_text(text, 4)

In [ ]:
SNRdB = 6
import base64
import cv2
_H = 300
_W = 200
img = cv2.imread("m.jpg")
img = cv2.resize(img, (_H, _W))
cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
img = img.reshape(_W*_H*3)
_bytes = base64.b64encode(img)
print(len(_bytes))
# _bytes = zlib.compress(_bytes)
# print(len(_bytes))
bits = ''.join(format(byte, '08b') for byte in _bytes)
decoded_bits, received_bits = send_bits(bits, SNRdB)
decoded_bytes = int(decoded_bits, 2).to_bytes((len(decoded_bits) + 7) // 8, 'big')
received_bits = int(received_bits, 2).to_bytes((len(received_bits) + 7) // 8, 'big')
# _bytes = zlib.decompress(received_bits)
img =base64.decodebytes(received_bits)
img = np.frombuffer(img, dtype=np.uint8)
img = img.reshape(_W,_H,3)

cv2.imshow("img", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
a = 2
Q = np.array([2127, 4309,  842, 1285, 3024 ,   0, 2181, 3467])
Q1 = permutation(Q, a)
print(Q1)
Q2 = permutation(Q, gl.reserve_e(a))
print(Q2)

In [ ]:
gl = GF([3,  1, 0])
for i in range(8):
    print(i, gl.reserve_e(i))